#Import lib

In [1]:
!pip install cleverhans

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 9.0 MB/s eta 0:00:00


In [2]:
import math
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import app, flags
from easydict import EasyDict
from tensorflow.keras import Model
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Conv2D, Input

from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
import pandas as pd


import matplotlib.pyplot as plt


## LOAD CIFAR-10 DATASET

In [3]:
def ld_cifar10():
    """Load training and test data."""

    def convert_types(image, label):
        image = tf.cast(image, tf.float32)
        image /= 127.5
        image -= 1.0
        return image, label

    dataset, info = tfds.load("cifar10", with_info=True, as_supervised=True)

    def augment_mirror(x):
        return tf.image.random_flip_left_right(x)

    def augment_shift(x, w=4):
        y = tf.pad(x, [[w] * 2, [w] * 2, [0] * 2], mode="REFLECT")
        return tf.image.random_crop(y, tf.shape(x))

    cifar10_train, cifar10_test = dataset["train"], dataset["test"]
    # Augmentation helps a lot in CIFAR10
    cifar10_train = cifar10_train.map(
        lambda x, y: (augment_mirror(augment_shift(x)), y)
    )
    cifar10_train = cifar10_train.map(convert_types).shuffle(10000).batch(128)
    cifar10_test = cifar10_test.map(convert_types).batch(1000)

    return EasyDict(train=cifar10_train, test=cifar10_test)

In [ ]:
# Variables
nb_epochs=50
#Les budgets sont en norme infini
adv_perturbation_budget=0.3
pgd_iterations=40
pgd_step_budget=0.01

adv_train=False #True pour faire la protection

data = ld_cifar10()
input_shape=(32,32,3)
num_classes=10
'''
def step_decay(epoch):
    initial_lr = 0.02
    drop = 0.96
    epochs_drop = 1
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr
'''
# Model Under Test

def model0(input_shape = input_shape):
    x=model = models.Sequential(name="VGG11_CIFAR10_simple")
    x=model.add(layers.Input(shape=input_shape))


    x=model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    x=model.add(layers.Activation('relu'))
    x=model.add(layers.BatchNormalization())

    x=model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    x=model.add(layers.Activation('relu'))
    x=model.add(layers.SpatialDropout2D(0.25))
    x=model.add(layers.BatchNormalization())
    x=model.add(layers.MaxPooling2D((2,2)))

    x=model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    x=model.add(layers.Activation('relu'))
    x=model.add(layers.BatchNormalization())

    x=model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    x=model.add(layers.Activation('relu'))
    x=model.add(layers.SpatialDropout2D(0.25))
    x=model.add(layers.BatchNormalization())
    x=model.add(layers.MaxPooling2D((2,2)))

    x=model.add(layers.Flatten())
    x=model.add(layers.Dense(10, activation='softmax'))
    return model

model = model0()
# Load training and test data


#model = Model(inputs, x)
model.summary()

loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(learning_rate=0.001)

#lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=0)

# Metrics to track the different accuracies.
train_loss = tf.metrics.Mean(name="train_loss")
train_acc_clean = tf.metrics.SparseCategoricalAccuracy()
test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
test_acc_pgd = tf.metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    predictions = model(x)
    loss = loss_object(y, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc_clean(y,predictions)

# Train model with adversarial training
for epoch in range(nb_epochs):
  # keras like pp of progress
  progress_bar_train = tf.keras.utils.Progbar(50000)
  for (x, y) in data.train:
    if adv_train:
      # Replace clean example with adversarial example for adversarial training
      x = projected_gradient_descent(model, x, adv_perturbation_budget, 0.01, 40, np.inf)
    train_step(x, y)
    progress_bar_train.add(x.shape[0], values=[("epoch",epoch),("Loss (train)", train_loss.result())])



Model: "VGG11_CIFAR10_simple"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_8             │ (None, 32, 32, 32)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_19 (Activation)      │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_9             │ (None, 16, 16, 32)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,642 (193.91 KB)

 Trainable params: 49,386 (192.91 KB)

 Non-trainable params: 256 (1.00 KB)

50000/50000 ━━━━━━━━━━━━━━━━━━━━ 22s 446us/step - epoch: 0.0000e+00 - Loss (train): 1.8077
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 13s 256us/step - epoch: 1.0000 - Loss (train): 1.5039
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 13s 254us/step - epoch: 2.0000 - Loss (train): 1.3626
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 12s 247us/step - epoch: 3.0000 - Loss (train): 1.2679
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 13s 253us/step - epoch: 4.0000 - Loss (train): 1.1988
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 13s 259us/step - epoch: 5.0000 - Loss (train): 1.1442
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 13s 267us/step - epoch: 6.0000 - Loss (train): 1.0997
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 12s 247us/step - epoch: 7.0000 - Loss (train): 1.0629
50000/50000 ━━━━━━━━━━━━━━━━━━━━ 13s 252us/step - epoch: 8.0000 - Loss (train): 1.0313
31744/50000 ━━━━━━━━━━━━━━━━━━━━ 5s 310us/step - epoch: 9.0000 - Loss (train): 1.0085

In [ ]:
# Evaluate on clean and adversarial data
progress_bar_test = tf.keras.utils.Progbar(10000)

# Lists to store results for a few examples
true_labels = []
clean_preds = []
fgsm_preds = []
pgd_preds = []
clean_confidences = []
fgsm_confidences = []
pgd_confidences = []

sample_count = 0
max_samples_to_display = 100  # Display results for up to 100 samples

for x, y in data.test:
  #print(np.shape(x))
  y_pred = model(x)
  test_acc_clean(y, y_pred)

  #ATTACKS WITH FGSM
  x_fgm = fast_gradient_method(model, x, adv_perturbation_budget, np.inf)
  y_pred_fgm = model(x_fgm)
  test_acc_fgsm(y, y_pred_fgm)

  #ATTACKS WITH PGD (iterative attack)
  x_pgd = projected_gradient_descent(model, x, adv_perturbation_budget, pgd_step_budget, pgd_iterations, np.inf)
  y_pred_pgd = model(x_pgd)
  test_acc_pgd(y, y_pred_pgd)

  # Store results for display
  if sample_count < max_samples_to_display:
      true_labels.extend(tf.get_static_value(y[:max_samples_to_display-sample_count]))

      clean_pred_values = tf.get_static_value(y_pred[:max_samples_to_display-sample_count])
      clean_preds.extend(tf.get_static_value(tf.argmax(clean_pred_values, axis=1)))
      clean_confidences.extend(tf.get_static_value(tf.reduce_max(tf.nn.softmax(clean_pred_values, axis=1), axis=1)))

      fgsm_pred_values = tf.get_static_value(y_pred_fgm[:max_samples_to_display-sample_count])
      fgsm_preds.extend(tf.get_static_value(tf.argmax(fgsm_pred_values, axis=1)))
      fgsm_confidences.extend(tf.get_static_value(tf.reduce_max(tf.nn.softmax(fgsm_pred_values, axis=1), axis=1)))

      pgd_pred_values = tf.get_static_value(y_pred_pgd[:max_samples_to_display-sample_count])
      pgd_preds.extend(tf.get_static_value(tf.argmax(pgd_pred_values, axis=1)))
      pgd_confidences.extend(tf.get_static_value(tf.reduce_max(tf.nn.softmax(pgd_pred_values, axis=1), axis=1)))


      sample_count += x.shape[0]


  #Print the clean accuracy (no attack) and the adversarial accuracy (i.e., the accuracy computed on the adversarial images)
  #NB: CIFAR-10 has 10 labels, the "random guess level" is accuracy=1/10=0.1
  progress_bar_test.add(x.shape[0])

print("\n--- Evaluation Results ---")
print("test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100))
print("test acc on FGM adversarial examples (%): {:.3f}".format(test_acc_fgsm.result() * 100)) #Utilisation du gradient pour genere une images (un mask)
print("test acc on PGD adversarial examples (%): {:.3f}".format(test_acc_pgd.result() * 100)) #La meme mais en iterative

# Display comparative table for a few examples
import pandas as pd

results_df = pd.DataFrame({
    'True Label': true_labels,
    'Clean Prediction': clean_preds,
    'Clean Confidence': clean_confidences,
    'FGSM Prediction': fgsm_preds,
    'FGSM Confidence': fgsm_confidences,
    'PGD Prediction': pgd_preds,
    'PGD Confidence': pgd_confidences
})

print("\n--- Comparative Table (First {} samples) ---".format(max_samples_to_display))
#display(results_df) # Commented out the display

# Save the table to a text file
adv_budget_str = str(adv_perturbation_budget).replace('.', '')
pgd_step_str = str(pgd_step_budget).replace('.', '')
if(adv_train==True):
  filename = f'./sample_data/protected_resultat_model2_adv_{adv_budget_str}_pgdstep_{pgd_step_str}.txt'
else:
  filename = f'./sample_data/resultat_model2_adv_{adv_budget_str}_pgdstep_{pgd_step_str}.txt'

with open(filename, 'w') as f:
    f.write("--- Evaluation Results ---\n")
    f.write("test acc on clean examples (%): {:.3f}\n".format(test_acc_clean.result() * 100))
    f.write("test acc on FGM adversarial examples (%): {:.3f}\n".format(test_acc_fgsm.result() * 100))
    f.write("test acc on PGD adversarial examples (%): {:.3f}\n".format(test_acc_pgd.result() * 100))
    f.write("\n--- Comparative Table (First {} samples) ---\n".format(max_samples_to_display))
    results_df.to_csv(f, sep='\t', index=False)

print(f"Comparative table saved to {filename}")


#SHOW THE ATTACK for a random sample
rand_sample_id=np.random.randint(np.shape(x)[0])

#We normalized our images into [0;1]. We make the inverse process to go into [0,255]
#NB: For the attacked images, the max perturbation is +/-0.05, so the images will be in [-0.05,1.05]
x_display = tf.cast(127.5*(x[rand_sample_id,:,:,:]+1.0),tf.uint8)
x_pgd_display = tf.cast(121.4*(x_pgd[rand_sample_id,:,:,:]+1.05),tf.uint8)
perturbation = x_pgd[rand_sample_id,:,:,:] - x[rand_sample_id,:,:,:]
print('Maximum perturbation: ',np.amax(perturbation))

#Display
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(6, 2), subplot_kw={'xticks': [], 'yticks': []})
axs.flat[0].imshow(tf.cast(x_display,tf.uint8))
axs.flat[0].set_title(str('clean, label=')+str(tf.get_static_value(y[rand_sample_id])))
axs.flat[1].imshow(tf.cast(x_pgd_display,tf.uint8))
axs.flat[1].set_title(str('pgd, label=')+str(np.argmax(tf.get_static_value(y_pred_pgd[rand_sample_id]))))
axs.flat[2].imshow(perturbation[:,:,0], cmap='gray')
axs.flat[2].set_title('Perturbation Mask')
plt.tight_layout()
plt.show()